## SQL Final Project

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product. 

### Description of the data
**books:**
<br>
Contains data on books:

   - **book_id**
   - **author_id**
   - **title**
   - **num_pages** — number of pages
   - **publication_date**
   - **publisher_id**
<br>

**authors:**
<br>
Contains data on authors:

   - **author_id**
   - **author**
<br>

**publishers:**
<br>
Contains data on publishers:

   - **publisher_id**
   - **publisher**
<br>

**ratings:**
<br>
Contains data on user ratings:

   - **rating_id**
   - **book_id**
   - **username** — the name of the user who rated the book
   - **rating**
<br>

**reviews:**
<br>
Contains data on customer reviews:

   - **review_id**
   - **book_id**
   - **username** — the name of the user who reviewed the book
   - **text** — the text of the review

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
Image(url= "https://pictures.s3.yandex.net/resources/Untitled_-_2020-07-02T142019.920_1593688954.png")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine


In [ ]:
pip install psycopg2-binary 

In [ ]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Study the tables 

In [ ]:
query="""
SELECT * 
FROM books 
LIMIT 5;
"""
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query="""
SELECT * 
FROM authors 
LIMIT 5;
"""
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query="""
SELECT * 
FROM publishers 
LIMIT 5;
"""
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query="""
SELECT * 
FROM ratings 
LIMIT 5;
"""
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query="""
SELECT * 
FROM reviews 
LIMIT 5;
"""
pd.io.sql.read_sql(query, con = engine)

### Conclusion:

We can understand that **books** will be our main table, connected by its primary key in a one to one (according to the drawings) connection. the **books** has 3 ID columns for **publisher**, **book** and **author**. Connected to **ratings** and **reviews** by **book_id**, to **publishers** by **publisher_id** and to **authors** by **authora_id**.

## Make an SQL query for each of the tasks

### Find the number of books released after January 1, 2000.

In [ ]:
query="""
SELECT
    DISTINCT(COUNT(title)) AS nomber_of_books
FROM 
   books
WHERE
   publication_date > '2000-01-01';
"""
pd.io.sql.read_sql(query, con = engine)

**There are 819 books released after January 1, 2000.**

### Find the number of user reviews and the average rating for each book.

**AVG rating**

In [ ]:
##query="""
##SELECT
    ##books.book_id,
    ##books.title,
    ##AVG(rating) AS average_rating,
    ##COUNT(DISTINCT(rating_id)) AS number_of_ratings
##FROM books
##INNER JOIN ratings USING(book_id)
##GROUP BY books.book_id
##ORDER BY  number_of_ratings DESC,average_rating DESC, title;
##"""
##pd.io.sql.read_sql(query, con = engine)

**AVG rating**

In [ ]:
##query="""
##SELECT
    ##books.book_id,
    ##books.title,
    ##AVG(rating) AS average_rating,
    ##COUNT(DISTINCT(review_id)) AS number_of_reviws
##FROM books
##INNER JOIN reviews USING(book_id)
##INNER JOIN ratings USING(book_id)
##GROUP BY books.book_id
##--ORDER BY  average_rating DESC,number_of_reviws DESC, title;
##"""
##pd.io.sql.read_sql(query, con = engine)

In [ ]:
query="""
SELECT
    re.book_id,
    title,
    re.review_count,
ra.avg_rating FROM books
LEFT JOIN (SELECT book_id, COUNT(text) AS review_count FROM reviews GROUP BY book_id) AS re ON books.book_id = re.book_id
LEFT JOIN (SELECT book_id, AVG(rating) AS avg_rating  FROM ratings GROUP BY book_id) AS ra ON re.book_id = ra.book_id
"""
pd.io.sql.read_sql(query, con = engine)

**We have 2 books with best performance, with average ratings with 4 and 5. Thoue A Dirty Job (Grim Reaper #1) and School's Out—Forever (Maximum Ride #2). There are 1000 books, we maybe miss ingormation about 6 for reviews. We have two options for reviews and rating counts.**

### Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [ ]:
query="""
SELECT
    books.publisher_id,
    publishers.publisher,
    COUNT(books.title) AS number_of_books
FROM publishers
INNER JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY books.publisher_id,publishers.publisher 
ORDER BY number_of_books DESC
LIMIT 1;
"""
pd.io.sql.read_sql(query, con = engine)

**The publisher that has released the greatest number of books with more than 50 pages is Penguin Books(id 212) with 42 books.**

### Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [ ]:
query="""
SELECT
    books.author_id,
    authors.author,
    books.title,
    AVG(ratings.rating) AS average_rating,
    COUNT(ratings.rating_id) AS rating_anount
FROM books
INNER JOIN authors ON authors.author_id = books.author_id
INNER JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.author_id,books.title,authors.author
HAVING COUNT(ratings.rating_id) >= 50
 
ORDER BY average_rating DESC
LIMIT 1;
"""
pd.io.sql.read_sql(query, con = engine)

**The author with the highest average book rating is J.K. Rowling/Mary GrandPré(id 236) with Harry Potter and the Prisoner of Azkaban having 4.414634 averege rating and 82 ratings published.**

### Find the average number of text reviews among users who rated more than 50 books.

In [ ]:
query="""
SELECT
    AVG(COUNT(DISTINCT(reviews.review_id))) OVER () AS averege_number_of_reviews
FROM reviews
INNER JOIN ratings ON reviews.username = ratings.username

GROUP BY ratings.username
HAVING COUNT(DISTINCT(ratings.rating_id)) > 50
 
LIMIT 1;
"""
pd.io.sql.read_sql(query, con = engine)

**Average number of text reviews among users who rated more than 50 books is 24.333333 rating per user.**

## Final Conclusion:

- **There are 821 books released after January 1, 2000.**
- **We have 2 books with best performance, with average ratings with 4 and 5. Thoue A Dirty Job (Grim Reaper #1) and School's Out—Forever (Maximum Ride #2). There are 1000 books, we maybe miss ingormation about 6 for reviews. We have two options for reviews and rating counts.**
- **The publisher  that has released the greatest number of books with more than 50 pages  is Penguin Books(id 212) with 42 books.**
- **The author with the highest average book rating is J.K. Rowling/Mary GrandPré(id 236) with Harry Potter and the Prisoner of Azkaban having 4.414634 averege rating and 82 ratings published.**
- **Average number of text reviews among users who rated more than 50 books is 24.333333 rating per user.**